Download Dataset from Kaggle

In [10]:
import kagglehub
import shutil

# Download latest version
path = kagglehub.dataset_download("andrewmvd/face-mask-detection")

print("Path to dataset files:", path)


# Step 2: Copy data to working directory and extract
dst_path = r"D:\AI\NTI AI\Face Mask Detection"
shutil.copytree(path, dst_path, dirs_exist_ok=True)
print("Copied to:", dst_path)

Resuming download from 234881024 bytes (182087059 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/andrewmvd/face-mask-detection?dataset_version_number=1 (234881024/416968083) bytes left.


100%|██████████| 398M/398M [04:49<00:00, 629kB/s] 

Extracting files...


Path to dataset files: C:\Users\lenovo\.cache\kagglehub\datasets\andrewmvd\face-mask-detection\versions\1
Copied to: D:\AI\NTI AI\Face Mask Detection


Dealing With GPU 

In [3]:
import torch
#clear cache
x=torch.cuda.empty_cache()
print(x)
# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)



None
Using device: cuda


Imports

In [4]:

import os
import xml.etree.ElementTree as ET
import shutil
import yaml
import random
from sklearn.model_selection import train_test_split
import cv2 
import numpy as np
from ultralytics import YOLO
from PIL import Image, ImageEnhance
import random

Setup Paths

In [5]:
base_dir = r"D:\AI\NTI AI\Face Mask Detection"
img_dir = os.path.join(base_dir, "images")
xml_dir = os.path.join(base_dir, "annotations")
aug_dir = os.path.join(base_dir, "augmented")
all_img_dir = os.path.join(base_dir, "images_all")
all_xml_dir = os.path.join(base_dir, "annotations_all")
preprocessed_dir = os.path.join(base_dir, "images_preprocessed")
yolo_dir = os.path.join(base_dir, "yolo_dataset_fixed")


Create Yolo Strcucture and Class Map

In [6]:
# CREATE DIRECTORIES
for folder in [aug_dir, all_img_dir, all_xml_dir, preprocessed_dir]:
    os.makedirs(folder, exist_ok=True)
    os.makedirs(os.path.join(folder, "images"), exist_ok=True)
    os.makedirs(os.path.join(folder, "annotations"), exist_ok=True)

for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(yolo_dir, "images", split), exist_ok=True)
    os.makedirs(os.path.join(yolo_dir, "labels", split), exist_ok=True)

# CLASS MAP
class_map = {"with_mask": 0, "without_mask": 1, "mask_weared_incorrect": 2}

Preprocessing Images

Data Augmentation

In [7]:

#  AUGMENT IMAGE
def augment_image(image_path):
    img = Image.open(image_path)
    if random.random() > 0.5:
        img = img.transpose(Image.FLIP_LEFT_RIGHT)
    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(random.uniform(0.7, 1.3))
    angle = random.randint(-10, 10)
    return img.rotate(angle)

# AUGMENT RARE CLASS
rare_class = "mask_weared_incorrect"
aug_factor = 3
image_files = [f for f in os.listdir(img_dir) if f.endswith(".png")]
for img_file in image_files:
    xml_file = img_file.replace(".png", ".xml")
    xml_path = os.path.join(xml_dir, xml_file)
    if not os.path.exists(xml_path):
        continue
    with open(xml_path) as f:
        if rare_class not in f.read():
            continue
    for i in range(aug_factor):
        new_img = augment_image(os.path.join(img_dir, img_file))
        aug_name = img_file.replace(".png", f"_aug{i}.png")
        new_img.save(os.path.join(aug_dir, "images", aug_name))
        shutil.copy(xml_path, os.path.join(aug_dir, "annotations", xml_file.replace(".xml", f"_aug{i}.xml")))

#  MERGE ORIGINAL + AUGMENTED
for src in [img_dir, os.path.join(aug_dir, "images")]:
    for f in os.listdir(src):
        shutil.copy(os.path.join(src, f), os.path.join(all_img_dir, f))

for src in [xml_dir, os.path.join(aug_dir, "annotations")]:
    for f in os.listdir(src):
        shutil.copy(os.path.join(src, f), os.path.join(all_xml_dir, f))


In [13]:

def preprocess_image(img_path, out_path, size=(640, 640)):
    img = cv2.imread(img_path)
    if img is None:
        print(f" Cannot read: {img_path}")
        return
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, size)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(out_path, img)

for img_file in os.listdir(all_img_dir):
    if not img_file.lower().endswith((".png", ".jpg", ".jpeg")):
        continue  # skip non-image files/folders
    src_path = os.path.join(all_img_dir, img_file)
    dst_path = os.path.join(preprocessed_dir, img_file)
    preprocess_image(src_path, dst_path)


Convert XML to Yolo Format

In [8]:
def xml_to_yolo(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)
    lines, classes = [], []
    for obj in root.findall("object"):
        cls = obj.find("name").text
        if cls not in class_map:
            continue
        bbox = obj.find("bndbox")
        xmin = int(float(bbox.find("xmin").text))
        ymin = int(float(bbox.find("ymin").text))
        xmax = int(float(bbox.find("xmax").text))
        ymax = int(float(bbox.find("ymax").text))
        x_center = ((xmin + xmax) / 2) / width
        y_center = ((ymin + ymax) / 2) / height
        w = (xmax - xmin) / width
        h = (ymax - ymin) / height
        lines.append(f"{class_map[cls]} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")
        classes.append(cls)
    return '\n'.join(lines), classes


Split Data and Save Splits

In [9]:
#  SPLIT DATA
all_data = []
image_files = [f for f in os.listdir(preprocessed_dir) if f.endswith(".png")]
for img_file in image_files:
    xml_path = os.path.join(all_xml_dir, img_file.replace(".png", ".xml"))
    if not os.path.exists(xml_path):
        continue
    yolo_txt, _ = xml_to_yolo(xml_path)
    if yolo_txt.strip():
        all_data.append((img_file, yolo_txt))

train_val, test = train_test_split(all_data, test_size=0.1, random_state=42)
train, val = train_test_split(train_val, test_size=2/9, random_state=42)

#  SAVE SPLITS

def save_split(data, split):
    for img_file, yolo_annots in data:
        shutil.copy(os.path.join(preprocessed_dir, img_file), os.path.join(yolo_dir, "images", split, img_file))
        label_file = img_file.replace(".png", ".txt")
        with open(os.path.join(yolo_dir, "labels", split, label_file), "w") as f:
            f.write(yolo_annots)

save_split(train, "train")
save_split(val, "val")
save_split(test, "test")
#Show Number of images and labels in each split
print("Data split and saved to YOLO format.")
print("Total images in Train:", len(os.listdir(os.path.join(yolo_dir, "images", "train"))))
print("Total labels:", len(os.listdir(os.path.join(yolo_dir, "labels", "train"))))     
print("Total images in Validation:", len(os.listdir(os.path.join(yolo_dir, "images", "val"))))
print("Total labels:", len(os.listdir(os.path.join(yolo_dir, "labels", "val"))))
print("Total images in Test:", len(os.listdir(os.path.join(yolo_dir, "images", "test"))))
print("Total labels:", len(os.listdir(os.path.join(yolo_dir, "labels", "test"))))

Data split and saved to YOLO format.
Total images in Train: 1041
Total labels: 1041
Total images in Validation: 528
Total labels: 528
Total images in Test: 293
Total labels: 293


Yolo works with Yaml-->so make Yaml Configuration

In [10]:
yaml_path = os.path.join(yolo_dir, "dataset.yaml")
yaml_content = {
    "train": "images/train",
    "val": "images/val",
    "test": "images/test",
    "nc": 3,
    "names": ["with_mask", "without_mask", "mask_weared_incorrect"]
}
with open(yaml_path, "w") as f:
    yaml.dump(yaml_content, f, sort_keys=False)


Train Yolo Model

In [13]:
from ultralytics import YOLO

# Try to load model safely, will auto-download if missing or corrupted
try:
    model = YOLO("yolov8n.pt")
except Exception as e:
    print("⚠️ YOLOv8n model file corrupted or missing. Downloading fresh copy...")
    model = YOLO("yolov8n")  # 'yolov8n' will trigger auto-download

# Train the model
results = model.train(
    data=yaml_path,
    epochs=50,
    batch=8,
    imgsz=640,
    device='cpu',  # better with GPU, but using CPU for problems in my device
    name="face_mask_augmented_preprocessed",
    optimizer="Adam",
    lr0=0.001,
    weight_decay=0.0005,
    cos_lr=True,
    lrf=0.01,
    augment=True,
    patience=10,
    save=True,
    save_period=1,
    val=True,
    plots=True
)


New https://pypi.org/project/ultralytics/8.3.160 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.70  Python-3.9.0 torch-2.6.0+cu118 CPU (12th Gen Intel Core(TM) i5-12450HX)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:\AI\NTI AI\Face Mask Detection\yolo_dataset_fixed\dataset.yaml, epochs=50, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=1, cache=False, device=cpu, workers=8, project=None, name=face_mask_augmented_preprocessed8, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, 

train: Scanning D:\AI\NTI AI\Face Mask Detection\yolo_dataset_fixed\labels\train.cache... 1041 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1041/1041 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning D:\AI\NTI AI\Face Mask Detection\yolo_dataset_fixed\labels\val.cache... 528 images, 0 backgrounds, 0 corrupt: 100%|██████████| 528/528 [00:00<?, ?it/s]


Plotting labels to C:\Users\lenovo\ultralytics\runs\detect\face_mask_augmented_preprocessed8\labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to C:\Users\lenovo\ultralytics\runs\detect\face_mask_augmented_preprocessed8
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50         0G       1.18      0.856      1.101          2        640: 100%|██████████| 131/131 [04:25<00:00,  2.03s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:41<00:00,  1.27s/it]

                   all        528       2842      0.857      0.696      0.766      0.529



50 epochs completed in 4.891 hours.
Optimizer stripped from C:\Users\lenovo\ultralytics\runs\detect\face_mask_augmented_preprocessed8\weights\last.pt, 6.2MB
Optimizer stripped from C:\Users\lenovo\ultralytics\runs\detect\face_mask_augmented_preprocessed8\weights\best.pt, 6.2MB

Validating C:\Users\lenovo\ultralytics\runs\detect\face_mask_augmented_preprocessed8\weights\best.pt...
Ultralytics 8.3.70  Python-3.9.0 torch-2.6.0+cu118 CPU (12th Gen Intel Core(TM) i5-12450HX)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [01:13<00:00,  2.24s/it]


                   all        528       2842      0.829      0.694      0.758      0.524
             with_mask        450       2159      0.886      0.816      0.863      0.608
          without_mask        195        499        0.8      0.721      0.764      0.514
 mask_weared_incorrect        147        184      0.801      0.543      0.649       0.45
Speed: 1.0ms preprocess, 114.0ms inference, 0.0ms loss, 8.2ms postprocess per image
Results saved to C:\Users\lenovo\ultralytics\runs\detect\face_mask_augmented_preprocessed8


Evaluate

IoU-->Intersection over Union=Area of Overlap/Area of Union
Overlap-->How much predicted box covers real object
Union=Area of predicted box+Area of Real Box-Overlap

Box Loss-->How tight Box is
CLS Loss-->How correct predicted class label is
DFL Loss-->How Confident the model is about the exact location of box edges

In [14]:
metrics = model.val(data=yaml_path, split="test")

print("✅ Test Evaluation Complete.")
print(f"mAP@50: {metrics.box.map50:.3f}")       # Good if > 0.7
print(f"mAP@50-95: {metrics.box.map:.3f}")      # Good if > 0.4
print(f"Mean Precision: {metrics.box.mp:.3f}") #Of models detection,How many are correct
print(f"Mean Recall: {metrics.box.mr:.3f}")     #of all real objects,how many did the model find

# Optional: Class-wise metrics
print("\n📊 Class-wise Precision, Recall, F1 Score:")
for i, name in metrics.names.items():
    p = metrics.box.p[i]
    r = metrics.box.r[i]
    f1 = metrics.box.f1[i]
    print(f"{name:<25} P={p:.3f}  R={r:.3f}  F1={f1:.3f}")


Ultralytics 8.3.70  Python-3.9.0 torch-2.6.0+cu118 CPU (12th Gen Intel Core(TM) i5-12450HX)


Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning D:\AI\NTI AI\Face Mask Detection\yolo_dataset_fixed\labels\test.cache... 293 images, 0 backgrounds, 0 corrupt: 100%|██████████| 293/293 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [01:03<00:00,  1.71s/it]


                   all        293       1949      0.817       0.66      0.709      0.489
             with_mask        262       1437      0.882      0.772      0.819      0.576
          without_mask        108        383      0.789      0.665      0.711      0.467
 mask_weared_incorrect         96        129      0.779      0.543      0.597      0.422
Speed: 1.3ms preprocess, 159.1ms inference, 0.0ms loss, 14.3ms postprocess per image
Results saved to C:\Users\lenovo\ultralytics\runs\detect\face_mask_augmented_preprocessed82
✅ Test Evaluation Complete.
mAP@50: 0.709
mAP@50-95: 0.489
Mean Precision: 0.817
Mean Recall: 0.660

📊 Class-wise Precision, Recall, F1 Score:
with_mask                 P=0.882  R=0.772  F1=0.823
without_mask              P=0.789  R=0.665  F1=0.722
mask_weared_incorrect     P=0.779  R=0.543  F1=0.640


Prediction Part

With Mask

The **confidence score** is a metric that represents how sure the model is that the detected object belongs to a specific class, ranging from 0 (not confident) to 1 (very confident).,It appears after Picture 


In [15]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Load the trained YOLO model
model = YOLO(r"C:\Users\lenovo\ultralytics\runs\detect\face_mask_detection5\weights\best.pt")

# Image path
image_path = r"D:\AI\NTI AI\images.jpg"

# Run prediction with confidence threshold
results = model.predict(source=image_path, save=True, conf=0.5)

# Plot using OpenCV
for result in results:
    image_with_boxes = result.plot()
    cv2.imshow("YOLOv8 Detection", image_with_boxes)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Plot using Matplotlib (optional)
plt.imshow(cv2.cvtColor(results[0].plot(), cv2.COLOR_BGR2RGB))
plt.title("Detection Results")
plt.axis("off")
plt.show()



image 1/1 D:\AI\NTI AI\images.jpg: 640x448 1 with_mask, 98.2ms
Speed: 6.3ms preprocess, 98.2ms inference, 408.1ms postprocess per image at shape (1, 3, 640, 448)
Results saved to C:\Users\lenovo\ultralytics\runs\detect\predict18


<Figure size 640x480 with 1 Axes>

Predict without mask

In [32]:
from ultralytics import YOLO
import cv2

# Load your trained model
model = YOLO(r"C:\Users\lenovo\ultralytics\runs\detect\face_mask_detection5\weights\best.pt")

# Predict on single image
image_path = r"D:\AI\NTI AI\images (1).jpg"
results = model.predict(source=image_path, save=True, conf=0.5)

# Show results (optional)
for result in results:
    cv2.imshow("Detection", result.plot())
    cv2.waitKey(0)
    cv2.destroyAllWindows()



image 1/1 D:\AI\NTI AI\images (1).jpg: 384x640 1 without_mask, 33.9ms
Speed: 3.0ms preprocess, 33.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to C:\Users\lenovo\ultralytics\runs\detect\predict15


Predict multi

In [16]:
from ultralytics import YOLO
import cv2

# Load your trained model
model = YOLO(r"C:\Users\lenovo\ultralytics\runs\detect\face_mask_detection5\weights\best.pt")

# Predict on single image
image_path = r"D:\AI\NTI AI\360_F_359987183_Hgu1X29BvLQ1t46CVXUVYDY4PPPHpvrw.jpg"
results = model.predict(source=image_path, save=True, conf=0.5)

# Show results (optional)
for result in results:
    cv2.imshow("Detection", result.plot())
    cv2.waitKey(0)
    cv2.destroyAllWindows()



image 1/1 D:\AI\NTI AI\360_F_359987183_Hgu1X29BvLQ1t46CVXUVYDY4PPPHpvrw.jpg: 448x640 1 with_mask, 76.5ms
Speed: 7.4ms preprocess, 76.5ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to C:\Users\lenovo\ultralytics\runs\detect\predict19


In [17]:
from ultralytics import YOLO
import cv2

# Load your trained model
model = YOLO(r"C:\Users\lenovo\ultralytics\runs\detect\face_mask_detection5\weights\best.pt")

# Predict on single image
image_path = r"D:\AI\NTI AI\images (2).jpg"
results = model.predict(source=image_path, save=True, conf=0.5)

# Show results (optional)
for result in results:
    cv2.imshow("Detection", result.plot())
    cv2.waitKey(0)
    cv2.destroyAllWindows()



image 1/1 D:\AI\NTI AI\images (2).jpg: 256x640 2 with_masks, 2 without_masks, 85.5ms
Speed: 6.3ms preprocess, 85.5ms inference, 6.4ms postprocess per image at shape (1, 3, 256, 640)
Results saved to C:\Users\lenovo\ultralytics\runs\detect\predict20


Using WebCam (Real Time)

In [18]:
from ultralytics import YOLO
import cv2
import time

def main():
    print("🟡 Loading model...")
    model_path = r"C:\Users\lenovo\ultralytics\runs\detect\face_mask_detection5\weights\best.pt"
    try:
        model = YOLO(model_path)
        print("✅ Model loaded successfully.")
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return

    print("🎥 Initializing webcam...")
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Error: Webcam not accessible.")
        return

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    time.sleep(1)  # Give camera some time to warm up

    print("🚀 Detection started. Press 'Q' to quit.")

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("⚠️ Failed to read frame. Retrying...")
                continue

            # Run detection
            results = model.predict(frame, conf=0.5, verbose=False)
            annotated_frame = frame  # fallback

            # Plot only if results exist
            if results and len(results[0].boxes) > 0:
                annotated_frame = results[0].plot()

                # Optional: print all detections
                for box in results[0].boxes:
                    cls_id = int(box.cls[0])
                    conf = float(box.conf[0])
                    label = model.names[cls_id]
                    print(f"🔍 Detected: {label} ({conf:.2f})")

            # Display
            cv2.imshow("Face Mask Detection (Press Q)", annotated_frame)

            # Exit on 'Q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except Exception as e:
        print(f"❗ Runtime error: {e}")

    finally:
        cap.release()
        cv2.destroyAllWindows()
        print("🛑 Program ended. Webcam released.")

if __name__ == "__main__":
    main()


🟡 Loading model...
✅ Model loaded successfully.
🎥 Initializing webcam...
🚀 Detection started. Press 'Q' to quit.
🔍 Detected: without_mask (0.80)
🔍 Detected: without_mask (0.84)
🔍 Detected: without_mask (0.83)
🔍 Detected: without_mask (0.82)
🔍 Detected: without_mask (0.83)
🔍 Detected: without_mask (0.81)
🔍 Detected: without_mask (0.78)
🔍 Detected: without_mask (0.80)
🔍 Detected: without_mask (0.83)
🔍 Detected: without_mask (0.84)
🔍 Detected: without_mask (0.85)
🔍 Detected: without_mask (0.86)
🔍 Detected: without_mask (0.87)
🔍 Detected: without_mask (0.84)
🔍 Detected: without_mask (0.85)
🔍 Detected: without_mask (0.86)
🔍 Detected: without_mask (0.87)
🔍 Detected: without_mask (0.86)
🔍 Detected: without_mask (0.86)
🔍 Detected: without_mask (0.86)
🔍 Detected: without_mask (0.87)
🔍 Detected: without_mask (0.84)
🔍 Detected: without_mask (0.85)
🔍 Detected: without_mask (0.85)
🔍 Detected: without_mask (0.85)
🔍 Detected: without_mask (0.84)
🔍 Detected: without_mask (0.85)
🔍 Detected: without_mas